In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt; plt.style.use('seaborn')
from sklearn import neighbors 
from sklearn.naive_bayes import GaussianNB

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
import warnings
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
import pandas as pd
import numpy as np

from matplotlib import pyplot

from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
data1 = pd.read_csv('Pathogen diagnostic model.txt',encoding='gbk')
data1

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
X1 =data1.iloc[0:543,3:9]
y =data1.iloc[0:543 ,9]
from numpy import *

X=X1.values
y

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(np.array(X),np.array(y), test_size=0.25,random_state = 0)

In [ ]:

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=5)

classifier =XGBClassifier(max_depth=4, learning_rate=0.01, n_estimators=500,gamma=0.25,colsample_bytree=1,min_child_weight = 1,subsample = 0.5,random_state=2018)  #Xgbc

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(X_train, y_train):
    probas_ = classifier.fit(X_train[train], y_train[train]).predict_proba(X_train[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_train[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")

plt.show()

In [ ]:
import shap
shap.initjs()
explainer = shap.Explainer(classifier)
shap_values = explainer(X1)
#列出前9个特征 shap.plots.bar(shap_values)

plt.figure(figsize=(4,4))
shap.summary_plot(shap_values, X1, plot_type="bar",show=False)

plt.savefig('Pathogen diagnostic model.PDF')

In [ ]:
#保存
#method 1：pickle
import pickle

with open('Pathogen diagnostic model.pickle','wb')as f: #python路径要用反斜杠
    pickle.dump(classifier,f) #将模型dump进f里面